# 머릿말

각각의 user / scene별로 별도의 csv파일로 저장하는걸 목적

user7-scene1 같은 경우는 둘로 나눠지기 때문에, 파일 두 개를 만든다.
user-scene-index 순서

In [1]:
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from math import pi
import pandas as pd

In [2]:
ds_split = {
    11: [(2000, 11000), (12000, 21000)],
    12: [(2000, 5000 ), (6000 , 10000)],
    13: [(2000, 10000), (11000, 18000)],
    21: [(2000, 11000), (12000, 20000)],
    22: [(2000, 6000 ), (7000 , 11000)],
    23: [(2000, 11000), (12000, 20000)],
    31: [(2000, 11000), (12000, 20000)],
    32: [(2000, 5000 ), (6000 , 11000)],
    33: [(2000, 10000), (11000, 19000)],
    41: [(2000, 10000), (11000, 19000)],
    42: [(2000, 6000 ), (7000 , 10000)],
    43: [(2000, 10000), (11000, 19000)],
    51: [(2000, 11000), (12000, 20000)],
    52: [(2000, 6000 ), (7000 , 10000)],
    53: [(2000, 10000), (11000, 19000)],
    61: [(2000, 9000 ), (10000, 17000)],
    62: [(2000, 11000), (12000, 20000)],
    63: [(2000, 10000), (11000, 19000)],
    71: [(2000, 7000 ), (9000 , 19000)],
    72: [(2000, 9000 ), (10000, 17000)],
    73: [(2000, 10000), (11000, 19000)]
}

### 각각의 User/Scene별로 데이터가 몇개씩 있는지

생각외로 고르게 분포되어있음

In [3]:
users_train, users_test = defaultdict(int), defaultdict(int)
scenes_train, scenes_test = defaultdict(int), defaultdict(int)

In [4]:
for key in ds_split.keys():
    user = key // 10
    scene = key % 10
    
    s, t = ds_split[key][0]
    users_train[user] += t - s
    scenes_train[scene] += t - s
    
    s, t = ds_split[key][1]
    users_test[user] += t - s
    scenes_test[scene] += t - s

In [5]:
users_train, users_test

(defaultdict(int,
             {1: 20000,
              2: 22000,
              3: 20000,
              4: 20000,
              5: 21000,
              6: 24000,
              7: 20000}),
 defaultdict(int,
             {1: 20000,
              2: 20000,
              3: 21000,
              4: 19000,
              5: 19000,
              6: 23000,
              7: 25000}))

In [6]:
scenes_train, scenes_test

(defaultdict(int, {1: 56000, 2: 34000, 3: 57000}),
 defaultdict(int, {1: 58000, 2: 34000, 3: 55000}))

## 데이터 생성

In [7]:
COLUMNS = ['timestamp', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'input_orientation_yaw', 'input_orientation_pitch', 'input_orientation_roll']

In [8]:
mkdir data/1116

In [37]:
all_len, all_yaw, all_pitch, all_roll, all_x, all_y, all_z = 0, 0, 0, 0, 0, 0, 0

for key in ds_split.keys():
    user = key // 10
    scene = key % 10
    csv_fname = f'data/org/motion_data_user{user}_scene{scene}.csv'
    csv = pd.read_csv(csv_fname)
    
    csv = csv[COLUMNS]
    
    # timestamp flick ==> sec
    csv.timestamp = (csv.timestamp - csv.timestamp[2000]) / 705600000
    
    # yaw, pitch, roll radian ==> degree
    csv.input_orientation_yaw *= 180 / pi
    csv.input_orientation_pitch *= 180 / pi
    csv.input_orientation_roll *= 180 / pi
    
    T = csv.timestamp
    dT = T[:-1].to_numpy() - T[1:].to_numpy()
    dT = np.insert(dT, 0, dT[0])
    
    # 가상 속도 / 가속도 생성 dyaw, ddyaw
    dyaw = np.insert(csv.input_orientation_yaw[:-1].to_numpy() - csv.input_orientation_yaw[1:].to_numpy(), 0, 0)
    dpitch = np.insert(csv.input_orientation_pitch[:-1].to_numpy() - csv.input_orientation_pitch[1:].to_numpy(), 0, 0)
    droll = np.insert(csv.input_orientation_roll[:-1].to_numpy() - csv.input_orientation_roll[1:].to_numpy(), 0, 0)
    ddyaw = np.insert(dyaw[:-1] - dyaw[1:], 0, 0) / dT
    ddpitch = np.insert(dpitch[:-1] - dpitch[1:], 0, 0) / dT
    ddroll = np.insert(droll[:-1] - droll[1:], 0, 0) / dT
    csv.insert(7, 'dyaw', dyaw)
    csv.insert(8, 'dpitch', dpitch)
    csv.insert(9, 'droll', droll)
    csv.insert(10, 'ddyaw', ddyaw)
    csv.insert(11, 'ddpitch', ddpitch)
    csv.insert(12, 'ddroll', ddroll)
    
    """ # 잘 안됨
    # 가상 xyz position 생성
    dx, dy, dz = [], [], []
    lv = 0
    for i, a in enumerate(csv.acceleration_x.to_numpy()):
        v = lv + a * dT[i]
        lv = v
        dx.append(v)
    lv = 0
    for i, a in enumerate(csv.acceleration_y.to_numpy()):
        v = lv + a * dT[i]
        lv = v
        dy.append(v)
    lv = 0
    for i, a in enumerate(csv.acceleration_z.to_numpy()):
        v = lv + a * dT[i]
        lv = v
        dz.append(v)
        
    x, y, z = [], [], []
    lw = 0
    for i, v in enumerate(dx):
        w = lw + v * dT[i]
        lw = w
        x.append(w)
    lw = 0
    for i, v in enumerate(dy):
        w = lw + v * dT[i]
        lw = w
        y.append(w)
    lw = 0
    for i, v in enumerate(dz):
        w = lw + v * dT[i]
        lw = w
        z.append(w)
    
    csv.insert(13, 'dx', dx)
    csv.insert(14, 'dy', dy)
    csv.insert(15, 'dz', dz)
    csv.insert(16, 'x', x)
    csv.insert(17, 'y', y)
    csv.insert(18, 'z', z)
    """
    
    for i, (S, T) in enumerate(ds_split[key]):
        dsv_fname = f'data/1116/user{user}_scene{scene}_{i}.csv'
        print(dsv_fname)
        dsv = csv.iloc[S:T]
        dsv.to_csv(dsv_fname, index=False)

data/1116/user1_scene1_0.csv
data/1116/user1_scene1_1.csv
data/1116/user1_scene2_0.csv
data/1116/user1_scene2_1.csv
data/1116/user1_scene3_0.csv
data/1116/user1_scene3_1.csv
data/1116/user2_scene1_0.csv
data/1116/user2_scene1_1.csv
data/1116/user2_scene2_0.csv
data/1116/user2_scene2_1.csv
data/1116/user2_scene3_0.csv
data/1116/user2_scene3_1.csv
data/1116/user3_scene1_0.csv
data/1116/user3_scene1_1.csv
data/1116/user3_scene2_0.csv
data/1116/user3_scene2_1.csv
data/1116/user3_scene3_0.csv
data/1116/user3_scene3_1.csv
data/1116/user4_scene1_0.csv
data/1116/user4_scene1_1.csv
data/1116/user4_scene2_0.csv
data/1116/user4_scene2_1.csv
data/1116/user4_scene3_0.csv
data/1116/user4_scene3_1.csv
data/1116/user5_scene1_0.csv
data/1116/user5_scene1_1.csv
data/1116/user5_scene2_0.csv
data/1116/user5_scene2_1.csv
data/1116/user5_scene3_0.csv
data/1116/user5_scene3_1.csv
data/1116/user6_scene1_0.csv
data/1116/user6_scene1_1.csv
data/1116/user6_scene2_0.csv
data/1116/user6_scene2_1.csv
data/1116/user

In [22]:
np.insert(dyaw, 0, 0).shape

(21472,)

In [23]:
dyaw.shape

(21471,)

In [38]:
csv

,timestamp,acceleration_x,acceleration_y,acceleration_z,input_orientation_yaw,input_orientation_pitch,input_orientation_roll,dyaw,dpitch,droll,ddyaw,ddpitch,ddroll
0,-283.848819,-0.069434,9.589070,-1.113338,-3.629473,7.595386,-1.410510,0.000000,0.000000,0.000000,-0.000000,-0.000000,-0.000000
1,-283.830711,-0.069434,9.601042,-1.101366,-3.595889,7.475031,-1.411981,-0.033583,0.120356,0.001471,-1.854568,6.646400,0.081221
2,-283.815272,-0.064645,9.622590,-1.087001,-3.539358,7.446354,-1.346180,-0.056531,0.028676,-0.065801,-1.486446,-5.938512,-4.357497
3,-283.798573,-0.045491,9.677659,-1.113338,-3.481707,7.329539,-1.311496,-0.057651,0.116816,-0.034684,-0.067029,5.277899,1.863317
4,-283.780946,-0.035914,9.720756,-1.072635,-3.452480,7.273142,-1.294892,-0.029227,0.056396,-0.016603,1.612555,-3.427759,1.025763
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19845,298.801921,-0.016760,9.761458,-0.529135,-1.018193,3.928019,-1.908800,0.106662,0.071004,0.050641,0.417961,2.498668,-2.433506
19846,298.820529,-0.131685,9.799767,-0.514769,-1.168380,3.917410,-1.962520,0.150188,0.010609,0.053721,2.339121,-3.245711,0.165484
19847,298.835512,-0.162811,9.818921,-0.495615,-1.254902,3.890114,-2.038934,0.086521,0.027296,0.076414,-4.249189,1.113753,1.514605
19848,298.852331,-0.284919,9.806950,-0.462095,-1.285986,3.847799,-2.102169,0.031084,0.042316,0.063235,-3.296125,0.893004,-0.783609


In [49]:
csv.iloc[:10][['timestamp', 'ddyaw']].to_numpy()

array([[-2.83848819e+02, -0.00000000e+00],
       [-2.83830711e+02, -1.85456787e+00],
       [-2.83815272e+02, -1.48644601e+00],
       [-2.83798573e+02, -6.70288650e-02],
       [-2.83780946e+02,  1.61255529e+00],
       [-2.83764215e+02, -1.86140534e+00],
       [-2.83748241e+02,  1.78265389e+00],
       [-2.83730287e+02,  1.54742643e+00],
       [-2.83714954e+02, -2.88931323e+00],
       [-2.83697515e+02,  3.71913675e+00]])